<a href="https://colab.research.google.com/github/GomathyDhanya/TF_Classifier/blob/master/NumberClassifierFromScratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

A Simple Number Classifier using deep learning techniques built with minimal use of keras functions

1. create model initialized with arbitrary weights and bias
2. forward pass : train model
3. optimize and update weights
4. test the model and get accuracy


Create model
1. create layers of weights and biases



In [ ]:
import tensorflow as tf
import math
from tensorflow.keras.datasets import mnist
import numpy as np


In [ ]:
class create_layer:
  def __init__(self, input_size, output_size, activation):
    self.activation=activation
    
    self.weights=tf.Variable(tf.random.uniform(shape=(input_size, output_size),minval=0,maxval=0.1))
    self.biases=tf.Variable(tf.zeros(shape=(output_size,1)))

  def __call__(self, inputs):
    return self.activation(tf.matmul(inputs, self.weights) + self.biases)

  @property
  def parameters(self):
    return [self.weights, self.biases]

2. Create Model blueprint

In [ ]:
class create_model:
  def __init__(self,layers):
    self.layers=layers
  
  def __call__(self,inputs):
    x=inputs
    for layer in self.layers:
      x=layer(x)
    return x

  @property 
  def parameters(self):
    parameters = []
    for layer in self.layers:
        parameters+=layer.parameters
    return parameters

  

3. instantiating a model

In [ ]:
model=create_model([create_layer(input_size=28 * 28, output_size=512, activation=tf.nn.relu),create_layer(input_size=512, output_size=10, activation=tf.nn.softmax)])


4. making 1 batch of images and targets

In [ ]:
class one_batch:
  def __init__(self,images,targets,batch_size=128):
    self.index = 0
    self.images = images
    self.targets = targets
    self.batch_size = batch_size
    self.num_batches = math.ceil(len(images) / batch_size)
  
  def get_images(self):
    images=self.images[self.index:self.index+self.batch_size]
    targets=self.targets[self.index:self.index+self.batch_size]
    self.index+=self.batch_size
    return images, targets


5. one forward pass and parameter update

In [ ]:
learning_rate = 1e-3 
  
def update_weights(gradients, weights):
    for g, w in zip(gradients, weights):
        w.assign_sub(g * learning_rate)


In [ ]:
def one_training_step(model, images_batch, targets_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        losses = tf.keras.losses.sparse_categorical_crossentropy(targets_batch, predictions)
        average_loss = tf.reduce_mean(losses)
    gradients = tape.gradient(average_loss, model.parameters)
    update_weights(gradients, model.parameters)
    return average_loss

6. running multiple rounds of training

In [ ]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        #print(f"Epoch {epoch_counter}")
        batch_generator = one_batch(images, labels)
        for batch_counter in range(batch_generator.num_batches):
            images_batch, labels_batch = batch_generator.get_images()
            loss = one_training_step(model, images_batch, labels_batch)
            #if batch_counter % 100 == 0:
                #print(f"loss at batch {batch_counter}: {loss:.2f}")

7. getting your training input and testing input

In [ ]:
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
  
train_images = train_images.reshape((60000, 28 * 28))
train_images = train_images.astype("float32") / 255  
test_images = test_images.reshape((10000, 28 * 28))
test_images = test_images.astype("float32") / 255 
  
fit(model, train_images, train_labels, epochs=10, batch_size=128)

8. Getting predictions and evaluating the test inputs

In [ ]:
predictions = model(test_images)
predictions = predictions.numpy()
predicted_labels = np.argmax(predictions, axis=1)
matches = predicted_labels == test_labels
print(f"accuracy: {matches.mean():.2f} %")

accuracy: 0.90 %


10. Openning Camera to capture hand written digit

In [ ]:
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode

def take_photo(filename='photo.jpg', quality=0.8):
  js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);

      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});

      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      await new Promise((resolve) => capture.onclick = resolve);

      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
  display(js)
  data = eval_js('takePhoto({})'.format(quality))
  binary = b64decode(data.split(',')[1])
  with open(filename, 'wb') as f:
    f.write(binary)
  return filename

In [ ]:
from IPython.display import Image
try:
  filename = take_photo()
  print('Saved to {}'.format(filename))
  display(Image(filename))
except Exception as err:
  print(str(err))

11. converting the image to 28x28 Binary image

In [ ]:
from PIL import Image, ImageChops

col = Image.open("/content/photo.jpg")
col=col.resize((28,28))
gray = col.convert('L')
bw = gray.point(lambda x: 0 if x<128 else 255, '1')
bw = ImageChops.invert(bw)
bw.save("result_bw.jpg")
imgArray = np.array(bw)
imgArray[imgArray > 0] = 1
print(imgArray.shape)

In [ ]:
imgArray=imgArray.reshape((1, 28 * 28))
imgArray = imgArray.astype("float32")
print(imgArray.shape)
print(imgArray)

12. Predicting the Number in the photo

In [ ]:
predict = model(imgArray)

tf.print(tf.math.argmax(predict,axis=1))